In [ ]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_colesterol = pd.read_csv('./datasets/dataset_colesterol.csv')

In [ ]:
df_colesterol.info()

In [ ]:
# Remover id
df_colesterol.drop(columns=['Id'], axis=1, inplace=True)

In [ ]:
# Renomear as colunas
df_colesterol.columns = [
  'grupo_sanguineo',
  'fumante',
  'nivel_atividade_fisica',
  'idade',
  'peso',
  'altura',
  'nivel_colesterol',
]

In [ ]:
# Copiar DF para DF EDA
df_colesterol_eda = df_colesterol.copy()

In [ ]:
# Copiar DF para DF EDA
df_colesterol_bucketing = df_colesterol.copy()

### EDA

In [ ]:
df_colesterol_eda.head()

In [ ]:
df_colesterol_eda.grupo_sanguineo.unique()

In [ ]:
df_colesterol_eda.isna().sum()

In [ ]:
# Medidas estatísticas
df_colesterol_eda.describe()

In [ ]:
# Coletar medikdas das variáveis categoricas
moda_grupo_sanguineo = df_colesterol_eda.grupo_sanguineo.mode()
moda_fumante = df_colesterol_eda.fumante.mode()
moda_nivel_atividade = df_colesterol_eda.nivel_atividade_fisica.mode()

In [ ]:
# Coletar medikdas das variáveis numéricas
mediana_idade = df_colesterol_eda.idade.median()
mediana_peso = df_colesterol_eda.peso.median()
mediana_altura = df_colesterol_eda.altura.median()

In [ ]:
df_colesterol_eda.fillna(value={'grupo_sanguineo': moda_grupo_sanguineo[0],
                                'fumante':moda_fumante[0],
                                'nivel_atividade_fisica':moda_nivel_atividade[0],
                                'idade': mediana_idade,
                                'peso':mediana_peso,
                                'altura':mediana_altura
                                }, inplace=True)

In [ ]:
df_colesterol_eda.idade = df_colesterol_eda.idade.astype(int)
df_colesterol_eda.altura = df_colesterol_eda.altura.astype(int)

In [ ]:
df_colesterol_eda.info()

In [ ]:
# Detectar outliers
sns.boxplot(data=df_colesterol_eda, x='peso')

In [ ]:
df_colesterol_eda[df_colesterol_eda['peso'] < 40].peso.count()

In [ ]:
# Remover
df_colesterol_eda.drop(df_colesterol_eda[df_colesterol_eda['peso'] < 40].index, axis=0, inplace=True)

In [ ]:
sns.boxplot(data=df_colesterol_eda, x='nivel_colesterol')

In [ ]:
# Cruzamento de variáveis com nível de colesterol
sns.boxplot(data=df_colesterol_eda, x='grupo_sanguineo', y='nivel_colesterol')

In [ ]:
# Cruzamento de variáveis com nível de colesterol
sns.boxplot(data=df_colesterol_eda, x='fumante', y='nivel_colesterol')

In [ ]:
# Cruzamento de variáveis com nível de colesterol
sns.boxplot(data=df_colesterol_eda, x='nivel_atividade_fisica', y='nivel_colesterol')

In [ ]:
# Cruzamento variáveis numéricas com nível colesterol
sns.scatterplot(data=df_colesterol_eda, x='idade',y='nivel_colesterol')

In [ ]:
# Cruzamento variáveis numéricas com nível colesterol
sns.scatterplot(data=df_colesterol_eda, x='peso',y='nivel_colesterol')

In [ ]:
# Cruzamento variáveis numéricas com nível colesterol
sns.scatterplot(data=df_colesterol_eda, x='altura',y='nivel_colesterol')

In [ ]:
sns.pairplot(df_colesterol_eda)

In [ ]:
# Converter variáveis categóricas em numéricas, ONE-HOT pandas
df_colesterol_eda = pd.get_dummies(df_colesterol_eda, columns=['grupo_sanguineo', 'fumante'], dtype='int64')

In [ ]:
df_colesterol_eda['nivel_atividade_fisica'] = pd.factorize(df_colesterol_eda.nivel_atividade_fisica)[0] + 1

In [ ]:
df_colesterol_eda.head()

In [ ]:
# Mapa de calor com correlação
plt.figure(figsize=(15,6))
sns.heatmap(df_colesterol_eda.corr(), vmin=-1, vmax=1, annot=True)

In [ ]:
# Correlação somente com variável target
sns.heatmap(df_colesterol_eda.corr()[['nivel_colesterol']].sort_values(by='nivel_colesterol', ascending=False),
             vmin=-1, vmax=1, annot=True, cmap='BrBG')

In [ ]:
# Bucketing Idade
bins_idade = [20,30, 40, 50, 60, 70, 80]
labels_idade = ['20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
df_colesterol_bucketing['escala_idade'] = pd.cut(x = df_colesterol_bucketing['idade'], bins = bins_idade, labels=labels_idade,
                                                 include_lowest=True)

In [ ]:
df_colesterol_bucketing.head(10)

In [ ]:
sns.boxplot(df_colesterol_bucketing, x='escala_idade', y='nivel_colesterol')

In [ ]:
# Bucketing Peso
bins_peso = [40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160]
labels_peso = ['40-49', '50-59', '60-69', '70-79', '80-89', '90-99', '100-109', 
               '110-119', '120-129', '130-139', '140-149', '150-159']
df_colesterol_bucketing['escala_peso'] = pd.cut(x = df_colesterol_bucketing['peso'], bins = bins_peso, labels=labels_peso,
                                                 include_lowest=True)

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(df_colesterol_bucketing, x='escala_peso', y='nivel_colesterol')

### Treinar Modelo

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
# Criar Dataset de Treino e Teste
df_colesterol.drop(df_colesterol[df_colesterol['peso'] < 40].index, axis=0, inplace=True)

In [ ]:
X = df_colesterol. drop(columns='nivel_colesterol', axis=1)
y = df_colesterol['nivel_colesterol']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.30, random_state=51)

In [ ]:
# Pipeline
# Imputar moda nas variáveis categóricas
# Padronizar variáveis numéricas
# OneHotEncode nas variáveis categoricas nominais
# OrdinalEncoder nas variáveis categóricas ordinais
# Imputar mediana nas variáveis numéricas 

# Nome das Colunas
colunas_categoricas = ['grupo_sanguineo', 'fumante']
colunas_numericas = ['idade', 'altura','peso']
colunas_ordinais = ['nivel_atividade_fisica']

In [ ]:
# Transformer Categoricas
transformer_categorias = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='most_frequent')),
  ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
# Transformer Ordinais
transformer_ordinais = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='most_frequent')),
  ('ordinal', OrdinalEncoder(categories=[['Baixo', 'Moderado', 'Alto']],handle_unknown='error'))
])

In [ ]:
# Transformer Numericas
transformer_numericas = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='median')),
  ('scaler', StandardScaler())
])

In [ ]:
# Criar um ColumnTransformer que encapsula todas as transformações
preprocessor = ColumnTransformer(
  transformers=[
    ('num', transformer_numericas, colunas_numericas),
    ('cat', transformer_categorias, colunas_categoricas),
    ('ord', transformer_ordinais, colunas_ordinais),
  ]
)

In [ ]:
# Criando o pipeline principal = Pre processamento + Treinamento
model_regr = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', LinearRegression())])

In [ ]:
# Treinar Modelo
model_regr.fit(X_train, y_train)

### Análise de Métricas

In [ ]:
# Gerar Predição
y_pred = model_regr.predict(X_test)

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
# Calcular MAE
mean_absolute_error(y_test, y_pred)

In [ ]:
# Calcular RMSE
mean_squared_error(y_test, y_pred, squared=False)

### Análise de resíduos

In [ ]:
residuos = y_test - y_pred

In [ ]:
# Transformar resíduos na escala padrão   -    (X -media) / desvio_padrao
from scipy.stats import zscore
residuos_std = zscore(residuos)

In [ ]:
# Verificar linearidade dos resíduos: Valores entre -2 e +2
# Verificar homocedasticidade: Valores em torno da reta
sns.scatterplot(x=y_pred, y=residuos_std)
plt.axhline(y=0)
plt.axhline(y=-2)
plt.axhline(y=2)

In [ ]:
# Chegar se resíduos seguem uma distribuição normal
# QQ Plot
import pingouin as pg
plt.figure(figsize=(14,8))
pg.qqplot(residuos_std, dist='norm', confidence=0.95)
plt.xlabel('Quantis Teóricos')
plt.ylabel('Resíduos na escala padrão')
plt.show()

In [ ]:
# Teste de normalidade de Shapiro-Wilk
from scipy.stats import shapiro, kstest, anderson
from statsmodels.stats.diagnostic import lilliefors, het_goldfeldquandt
stat_shapiro, p_value_shapiro = shapiro(residuos)
print('Estatística do Teste: {} e P-Value: {}'.format(stat_shapiro, p_value_shapiro))

In [ ]:
# Teste de Kolmogorov-Sminorv
stat_ks, p_value_ks = kstest(residuos, 'norm')
print('Estatística do Teste: {} e P-Value: {}'.format(stat_ks, p_value_ks))

In [ ]:
# Teste de Lilliefors
stat_ll, p_value_ll = lilliefors(residuos, dist='norm', pvalmethod='table')
print('Estatística do Teste: {} e P-Value: {}'.format(stat_ll, p_value_ll))

In [ ]:
# Teste de Anderson-darling
stat_and, critical_and, significance_and = anderson(residuos, dist='norm')

In [ ]:
critical_and

In [ ]:
significance_and

In [ ]:
print('Estatística do Teste: {} e valor crítico: {}'.format(stat_and, critical_and[2]))

In [ ]:
# Teste de Homocedasticidade de Goldfeld-Quandt
pipe = Pipeline(steps=[['preprocessor', preprocessor]])
X_test_transformed = pipe.fit_transform(X_test)

In [ ]:
X_test_transformed

In [ ]:
test_goldfeld = het_goldfeldquandt(residuos, X_test_transformed)
stat_goldfeld = test_goldfeld[0]
p_value = test_goldfeld[1]
print('Estatística do Teste: {} e P-Value: {}'.format(stat_goldfeld, p_value))

### Predições individuais do modelelo

In [ ]:
predicao_individual = {
  'grupo_sanguineo' : 'O',
  'fumante' : 'Não',
  'nivel_atividade_fisica' : 'Alto',
  'idade' : 40,
  'peso' : 70,
  'altura' : 180,
}
sample_df = pd.DataFrame(predicao_individual, index=[1])

In [531]:
# Predição
model_regr.predict(sample_df)

array([164.43571502])

In [ ]:
# Salvar Modelo
import joblib
joblib.dump(model_regr, './modelo_colesterol.pkl')